In [20]:
import requests, json

def get_all_bybit_spot_pairs():
    
    url = "https://api.bybit.com/v5/market/instruments-info"

    params = {"category": "spot", "limit": 500}
    all_pairs = []

    while True:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        result = response.json()["result"]

        all_pairs.extend(result["list"])  

        cursor = result.get("nextPageCursor")
        if not cursor:
            break
        params["cursor"] = cursor

    return all_pairs

bybit_data = get_all_bybit_spot_pairs()

In [ ]:
jupiter_url = "https://lite-api.jup.ag/tokens/v2/tag?query=verified"
jupiter_data = requests.get(jupiter_url).json()


# Build symbol-to-mint mapping
symbol_to_mints = {}
for token in jupiter_data:
    symbol = token["symbol"]
    if symbol not in symbol_to_mints:
        symbol_to_mints[symbol] = []
    symbol_to_mints[symbol].append({
        "mint": token["id"],  
        "name": token.get("name", ""),
        "decimals": token.get("decimals", 6)
    })

print(symbol_to_mints)


{'SOL': [{'mint': 'So11111111111111111111111111111111111111112', 'name': 'Wrapped SOL', 'decimals': 9}], 'USDC': [{'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'name': 'USD Coin', 'decimals': 6}], 'TRUMP': [{'mint': '6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN', 'name': 'OFFICIAL TRUMP', 'decimals': 6}, {'mint': 'HaP8r3ksG76PhQLTqR8FYBeNiQpejcFbQmiHbg787Ut1', 'name': 'MAGA (Wormhole)', 'decimals': 8}, {'mint': 'AwRErBEFGTnohzfLeRSBH9HddQEy2oeRxnWLrbvFFh95', 'name': 'TRUMP', 'decimals': 6}, {'mint': '4h8LjZWUfUQVgbEZ29UzTuGXNW6rwrJis78ZU66ekkPV', 'name': 'Donald Trump', 'decimals': 9}, {'mint': 'BqhNdGtS1Nqtzi2MvZ7G8NN1vRuHZ12UpHGJKe71e1JT', 'name': 'MoonTrump', 'decimals': 9}], 'USDT': [{'mint': 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB', 'name': 'USDT', 'decimals': 6}], 'jlUSDC': [{'mint': '9BEcn9aPEmhSPbPQeFGjidRiEKki46fVQDyPpSQXPA2D', 'name': 'jupiter lend USDC', 'decimals': 6}], 'JLP': [{'mint': '27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4', 'name': 'Jupiter Perps'

In [22]:
# Build trading pairs
trading_pairs = []
for pair in bybit_data:
    base = pair["baseCoin"]
    quote = pair["quoteCoin"]
    
    if base in symbol_to_mints and quote in symbol_to_mints:
        trading_pairs.append({
            "bybit_symbol": pair["symbol"],
            "base_token": {
                "symbol": base,
                "jupiter_mints": symbol_to_mints[base]
            },
            "quote_token": {
                "symbol": quote,
                "jupiter_mints": symbol_to_mints[quote]
            }
        })

# Save
with open("monitor.json", "w") as f:
    json.dump(trading_pairs, indent=2, fp=f)

print(f"Found {len(trading_pairs)} pairs")

Found 157 pairs
